In [21]:
%pip install -q global-land-mask pyproj kaleido

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from global_land_mask import globe
from pyproj import Geod
import plotly.io as pio

GEOD = Geod(ellps="WGS84")

In [ ]:
def land_parallel_length(
    lat_deg: float,
    step: float = 0.01,
) -> tuple[float, float]:
    """
    Суммарная длина суши вдоль параллели.

    Возвращает
        (total_km_land, total_km)
    где
        total_km_land – длина сухопутных участков параллели, км,
        total_km – полная длина параллели (при данном step), км.

    Parameters
    ----------
    lat_deg : float
        Широта параллели (–90 … +90 °).
    step : float, optional
        Шаг по долготе, °. 0.01° ≈ 1.1 км на экваторе.
        (должен быть 0 < step ≤ 1).
    """
    if not -90.0 <= lat_deg <= 90.0:
        raise ValueError("Широта должна быть в диапазоне –90…+90°")
    if step <= 0 or step > 1:
        raise ValueError("step должен быть > 0 и ≤ 1°")

    # Равномерные долготы, –180…+180 включительно
    n = int(round(360 / step)) + 1
    lons = np.linspace(-180.0, 180.0, n)

    # Маска суши (True = суша, False = вода)
    land_mask = globe.is_land(np.full_like(lons, lat_deg), lons)

    total_m = 0.0
    total_m_land = 0.0

    # Идём по каждой паре соседних долгот
    for i in range(len(lons) - 1):
        lon1, lon2 = lons[i], lons[i + 1]

        # Геодезическая длина маленького отрезка
        _, _, seg = GEOD.inv(lon1, lat_deg, lon2, lat_deg)
        seg = abs(seg)          # метры
        total_m += seg

        # Считаем сушу, только если обе точки сегмента – суша
        if land_mask[i] and land_mask[i + 1]:
            total_m_land += seg

    return total_m_land / 1_000.0, total_m / 1_000.0

In [ ]:
from tqdm.auto import tqdm

latitudes = list(range(-90, 91))
latitudes_land_lengths = []
lattitudes_lengths = []

for lat in tqdm(latitudes):
    land_km, full_km = land_parallel_length(lat, step=0.01)
    latitudes_land_lengths.append(land_km)
    lattitudes_lengths.append(full_km)

  0%|          | 0/181 [00:00<?, ?it/s]

In [8]:
import plotly.express as px

fig = px.line(
    x=latitudes,
    y=latitudes_land_lengths,
    labels={
        "x": "Широта, °",
        "y": "Длина суши вдоль параллели, км",
    },
)
fig.update_layout(
    title="Сколько суши приходится на каждую параллель",
    xaxis_title="Широта (°)",
    yaxis_title="Сухопутная длина параллели (км)"
)
fig.show()
pio.write_image(
    fig, 
    "latitude_land_len_line.png", 
    "png", 
    scale=2.0, 
    width=1280, 
    height=480
)

In [33]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

df = pd.DataFrame({"lat": latitudes, "land_km": latitudes_land_lengths})
df = df.sort_values("lat")
df["lon"] = 0  # кладём всё на меридиан 0°

# Настраиваем размер маркеров пропорционально длине суши
# Множитель можно регулировать для лучшей визуализации
# Так как здесь используются абсолютные значения в км, нужно подобрать подходящий коэффициент
marker_size = df["land_km"] / max(lattitudes_lengths) * 30

fig_geo = go.Figure()

fig_geo.add_trace(go.Scattergeo(
    lon=df["lon"],
    lat=df["lat"],
    mode="lines+markers",
    marker=dict(
        size=marker_size,  # Теперь размер точек пропорционален длине суши
        color=df["land_km"],
        colorbar=dict(title="Длина суши (км)"),
        colorscale="Viridis",  # Добавляем цветовую шкалу
    ),
    line=dict(width=2, color="rgba(0,0,0,0.5)"),
    name="Длина суши по параллелям"
))

fig_geo.update_geos(
    projection_type="natural earth",
    showcoastlines=True,
    showcountries=True,
    showland=True,
    landcolor="lightgray"
)

fig_geo.update_layout(
    title="Сколько суши приходится на каждую параллель",
    height=600,
    width=800
)

fig_geo.show()
pio.write_image(
    fig_geo, 
    "latitude_land_len_globe.png", 
    "png", 
    scale=1.0, 
    width=1280, 
    height=720
)

In [32]:
# То же, но в процентах (ведь длина окружности вдоль широты не постоянна!)
import plotly.express as px

lattitudes_land_rel = np.array(latitudes_land_lengths) / (0.001 + np.array(lattitudes_lengths))

fig = px.line(
    x=latitudes,
    y=lattitudes_land_rel,
    labels={
        "x": "Широта, °",
        "y": "Длина суши вдоль параллели, (отн.)",
    },
)
fig.update_layout(
    title="Сколько суши приходится на каждую параллель",
    xaxis_title="Широта (°)",
    yaxis_title="Сухопутная длина параллели (отн.)"
)
fig.show()

pio.write_image(
    fig, 
    "latitude_land_len_line_rel.png", 
    "png", 
    scale=2.0, 
    width=1280, 
    height=480
)

In [30]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

df = pd.DataFrame({"lat": latitudes, "land_km": lattitudes_land_rel})
df = df.sort_values("lat")
df["lon"] = 0  # кладём всё на меридиан 0°

marker_size = df["land_km"] * 30 

fig_geo = go.Figure()

fig_geo.add_trace(go.Scattergeo(
    lon=df["lon"],
    lat=df["lat"],
    mode="lines+markers",
    marker=dict(
        size=marker_size,  # Теперь размер точек пропорционален длине суши
        color=df["land_km"],
        colorbar=dict(title="Длина суши (отн.)"),
        colorscale="Viridis",
    ),
    line=dict(width=2, color="rgba(0,0,0,0.5)"),
    name="Длина суши по параллелям"
))

fig_geo.update_geos(
    projection_type="natural earth",
    showcoastlines=True,
    showcountries=True,
    showland=True,
    landcolor="lightgray"
)

fig_geo.update_layout(
    title="Сколько суши приходится на каждую параллель",
    height=600,
    width=800
)

fig_geo.show()

pio.write_image(
    fig_geo, 
    "latitude_land_len_globe_rel.png", 
    "png", 
    scale=1.0, 
    width=1280, 
    height=720
)